In [1]:
import warnings
warnings.filterwarnings("ignore")
import intake

In [2]:
col = intake.open_esm_metadatastore(collection_input_definition="CESM1-LE", overwrite_existing=True)

Getting file listing: CTRL:GLADE:posix:/glade/collections/cdg/data/cesmLE/CESM-CAM5-BGC-LE


Filename : f.e11.FAMIPC5CN.f09_f09.rcp85.toga.ens10.clm2.h0.THBOT.200601-201412.nc_temp_.nc does not conform to expected pattern
Filename : f.e11.FAMIPC5CN.f09_f09.rcp85.toga.ens10.clm2.h0.ERRH2OSNO.200601-201412.nc_temp_.nc does not conform to expected pattern
Filename : f.e11.FAMIPC5CN.f09_f09.rcp85.toga.ens10.clm2.h0.PSNSUN.200601-201412.nc_temp_.nc does not conform to expected pattern
Filename : f.e11.FAMIPC5CN.f09_f09.rcp85.toga.ens10.clm2.h0.PRODUCT_CLOSS.200601-201412.nc_temp_.nc does not conform to expected pattern
Filename : f.e11.FAMIPC5CN.f09_f09.rcp85.toga.ens10.clm2.h0.PSNSUN_TO_CPOOL.200601-201412.nc_temp_.nc does not conform to expected pattern

Getting file listing: CTRL:HPSS:hsi:/CCSM/csm/CESM-CAM5-BGC-LE
Getting file listing: 20C:GLADE:posix:/glade/collections/cdg/data/cesmLE/CESM-CAM5-BGC-LE


Filename : f.e11.FAMIPC5CN.f09_f09.rcp85.toga.ens10.clm2.h0.THBOT.200601-201412.nc_temp_.nc does not conform to expected pattern
Filename : f.e11.FAMIPC5CN.f09_f09.rcp85.toga.ens10.clm2.h0.ERRH2OSNO.200601-201412.nc_temp_.nc does not conform to expected pattern
Filename : f.e11.FAMIPC5CN.f09_f09.rcp85.toga.ens10.clm2.h0.PSNSUN.200601-201412.nc_temp_.nc does not conform to expected pattern
Filename : f.e11.FAMIPC5CN.f09_f09.rcp85.toga.ens10.clm2.h0.PRODUCT_CLOSS.200601-201412.nc_temp_.nc does not conform to expected pattern
Filename : f.e11.FAMIPC5CN.f09_f09.rcp85.toga.ens10.clm2.h0.PSNSUN_TO_CPOOL.200601-201412.nc_temp_.nc does not conform to expected pattern

Getting file listing: 20C:HPSS:hsi:/CCSM/csm/CESM-CAM5-BGC-LE
Getting file listing: RCP85:GLADE:posix:/glade/collections/cdg/data/cesmLE/CESM-CAM5-BGC-LE


Filename : f.e11.FAMIPC5CN.f09_f09.rcp85.toga.ens10.clm2.h0.THBOT.200601-201412.nc_temp_.nc does not conform to expected pattern
Filename : f.e11.FAMIPC5CN.f09_f09.rcp85.toga.ens10.clm2.h0.ERRH2OSNO.200601-201412.nc_temp_.nc does not conform to expected pattern
Filename : f.e11.FAMIPC5CN.f09_f09.rcp85.toga.ens10.clm2.h0.PSNSUN.200601-201412.nc_temp_.nc does not conform to expected pattern
Filename : f.e11.FAMIPC5CN.f09_f09.rcp85.toga.ens10.clm2.h0.PRODUCT_CLOSS.200601-201412.nc_temp_.nc does not conform to expected pattern
Filename : f.e11.FAMIPC5CN.f09_f09.rcp85.toga.ens10.clm2.h0.PSNSUN_TO_CPOOL.200601-201412.nc_temp_.nc does not conform to expected pattern

Getting file listing: RCP85:HPSS:hsi:/CCSM/csm/CESM-CAM5-BGC-LE
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110702 entries, 0 to 110701
Data columns (total 18 columns):
resource            110702 non-null object
resource_type       110702 non-null object
direct_access       110702 non-null object
experiment          110702 

In [3]:
col.df.head()

,resource,resource_type,direct_access,experiment,case,component,stream,variable,date_range,member_id,file_fullpath,file_basename,file_dirname,ctrl_branch_year,year_offset,sequence_order,has_ocean_bgc,grid
0,CTRL:GLADE:posix:/glade/collections/cdg/data/c...,posix,True,CTRL,b.e11.B1850C5CN.f09_g16.005,atm,cam.h1,T200,18000101-18991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,b.e11.B1850C5CN.f09_g16.005.cam.h1.T200.180001...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,1448.0,0,True,NaN
1,CTRL:GLADE:posix:/glade/collections/cdg/data/c...,posix,True,CTRL,b.e11.B1850C5CN.f09_g16.005,atm,cam.h1,T200,08000101-08991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,b.e11.B1850C5CN.f09_g16.005.cam.h1.T200.080001...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,1448.0,0,True,NaN
2,CTRL:GLADE:posix:/glade/collections/cdg/data/c...,posix,True,CTRL,b.e11.B1850C5CN.f09_g16.005,atm,cam.h1,T200,04020101-04991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,b.e11.B1850C5CN.f09_g16.005.cam.h1.T200.040201...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,1448.0,0,True,NaN
3,CTRL:GLADE:posix:/glade/collections/cdg/data/c...,posix,True,CTRL,b.e11.B1850C5CN.f09_g16.005,atm,cam.h1,T200,10000101-10991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,b.e11.B1850C5CN.f09_g16.005.cam.h1.T200.100001...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,1448.0,0,True,NaN
4,CTRL:GLADE:posix:/glade/collections/cdg/data/c...,posix,True,CTRL,b.e11.B1850C5CN.f09_g16.005,atm,cam.h1,T200,11000101-11991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,b.e11.B1850C5CN.f09_g16.005.cam.h1.T200.110001...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,1448.0,0,True,NaN


In [4]:
col.df.resource_type.unique()

array(['posix'], dtype=object)

In [5]:
col.df.component.unique()

array(['atm', 'ocn', 'lnd', 'rof', 'ice'], dtype=object)

In [6]:
variables = col.df.variable.unique()

In [7]:
def print_info(cat):
    query_results = cat.query_results
    print(f"There are/is {query_results.variable.nunique()} unique variables: {query_results.variable.unique()}\n\n")
    print(f"Number of files: {query_results.file_fullpath.nunique()}\n\n")
    print(f"Date ranges: {query_results.date_range.unique()}\n\n")


## Variables to include in AWS

### Atmosphere, monthly, 3D

In [8]:
atmos_mon_3d = ["T", "U", "V", "Q", "Z3"]

In [9]:
cat = col.search(variable=atmos_mon_3d, component="atm", stream="cam.h0")
print_info(cat)

There are/is 5 unique variables: ['Q' 'T' 'U' 'V' 'Z3']


Number of files: 655


Date ranges: ['040001-049912' '050001-059912' '060001-069912' '070001-079912'
 '080001-089912' '090001-099912' '100001-109912' '110001-119912'
 '120001-129912' '130001-139912' '140001-149912' '150001-159912'
 '160001-169912' '170001-179912' '180001-189912' '190001-199912'
 '200001-209912' '210001-220012' '185001-200512' '192001-200512'
 '200601-208012' '208101-210012' '200601-210012']




**NOTE:**
    
- There are **655** files. The spreadsheet mentions **1010** files. 

### Atmosphere, Monthly, Surface

In [10]:
atmos_mon_surface = ["FLNS", "FLNSC", "FLUT", "FSNS", "FSNSC", "FSNTOA", "ICEFRAC", 
                     "LHFLX", "PRECC", "PRECL", "PRECSC", "PRECSL", "PSL", "SHFLX", 
                     "TMQ", "TREFHT", "TREFHTMN", "TREFHTMX", "TS"]

In [11]:
cat = col.search(variable=atmos_mon_surface, component="atm", stream="cam.h0")
print_info(cat)

There are/is 19 unique variables: ['FLNS' 'FLNSC' 'FLUT' 'FSNS' 'FSNSC' 'FSNTOA' 'ICEFRAC' 'LHFLX' 'PRECC'
 'PRECL' 'PRECSC' 'PRECSL' 'PSL' 'SHFLX' 'TMQ' 'TREFHT' 'TREFHTMN'
 'TREFHTMX' 'TS']


Number of files: 2490


Date ranges: ['040001-049912' '050001-059912' '060001-069912' '070001-079912'
 '080001-089912' '090001-099912' '100001-109912' '110001-119912'
 '120001-129912' '130001-139912' '140001-149912' '150001-159912'
 '160001-169912' '170001-179912' '180001-189912' '190001-199912'
 '200001-209912' '210001-220012' '185001-200512' '192001-200512'
 '200601-208012' '208101-210012' '200601-210012']




**NOTE:**
    
- There are **2490** files. The spreadsheet mentions **3839** files. 

### Land, Monthly

In [12]:
land_vars = ["FSNO", "H2OSNO", "QRUNOFF", "RAIN", "SNOW", "SOILLIQ", "SOILWATER_10CM"]

In [13]:
cat = col.search(variable=land_vars, component="lnd", stream="clm2.h0")
print_info(cat)

There are/is 7 unique variables: ['FSNO' 'H2OSNO' 'QRUNOFF' 'RAIN' 'SNOW' 'SOILLIQ' 'SOILWATER_10CM']


Number of files: 917


Date ranges: ['040001-049912' '050001-059912' '060001-069912' '070001-079912'
 '080001-089912' '090001-099912' '100001-109912' '110001-119912'
 '120001-129912' '130001-139912' '140001-149912' '150001-159912'
 '160001-169912' '170001-179912' '180001-189912' '190001-199912'
 '200001-209912' '210001-220012' '185001-200512' '192001-200512'
 '200601-208012' '208101-210012' '200601-210012']




### Land, Daily

In [14]:
cat = col.search(variable=land_vars, component="lnd", stream="clm2.h1")
print_info(cat)

There are/is 6 unique variables: ['FSNO' 'H2OSNO' 'QRUNOFF' 'RAIN' 'SNOW' 'SOILWATER_10CM']


Number of files: 786


Date ranges: ['04030101-04991231' '05000101-05991231' '06000101-06991231'
 '07000101-07991231' '08000101-08991231' '09000101-09991231'
 '10000101-10991231' '11000101-11991231' '12000101-12991231'
 '13000101-13991231' '14000101-14991231' '15000101-15991231'
 '16000101-16991231' '17000101-17991231' '18000101-18991231'
 '19000101-19991231' '20000101-20991231' '21000101-22001231'
 '18500101-20051231' '19200101-20051231' '20060101-20801231'
 '20810101-21001231' '20060101-21001231']




**NOTE:**
    
- There are **786+917=1703** files. The spreadsheet mentions **1414** files. 

### Ocean, Monthly, Surface

In [15]:
ocn_surface_vars = ["SST", "SSH"]

In [16]:
cat = col.search(variable=ocn_surface_vars, component="ocn", stream="pop.h")
print_info(cat)

There are/is 2 unique variables: ['SSH' 'SST']


Number of files: 262


Date ranges: ['040001-049912' '050001-059912' '060001-069912' '070001-079912'
 '080001-089912' '090001-099912' '100001-109912' '110001-119912'
 '120001-129912' '130001-139912' '140001-149912' '150001-159912'
 '160001-169912' '170001-179912' '180001-189912' '190001-199912'
 '200001-209912' '210001-220012' '185001-200512' '192001-200512'
 '200601-208012' '208101-210012' '200601-210012']




### Ocean, Monthly, 3D

In [17]:
ocn_3d_vars = ["SALT"]
cat = col.search(variable=ocn_3d_vars, component="ocn", stream="pop.h")
print_info(cat)

There are/is 1 unique variables: ['SALT']


Number of files: 131


Date ranges: ['040001-049912' '050001-059912' '060001-069912' '070001-079912'
 '080001-089912' '090001-099912' '100001-109912' '110001-119912'
 '120001-129912' '130001-139912' '140001-149912' '150001-159912'
 '160001-169912' '170001-179912' '180001-189912' '190001-199912'
 '200001-209912' '210001-220012' '185001-200512' '192001-200512'
 '200601-208012' '208101-210012' '200601-210012']




**NOTE:**
    
- There are **262+131=393** files. The spreadsheet mentions **438** files. 

### Ice, Monthly

In [18]:
ice_mon_vars = ["aice_nh", "aice_sh", "hi_nh", "hi_sh"]
cat = col.search(variable=ice_mon_vars, component="ice", stream="cice.h")
print_info(cat)

There are/is 4 unique variables: ['aice_nh' 'aice_sh' 'hi_nh' 'hi_sh']


Number of files: 524


Date ranges: ['040001-049912' '050001-059912' '060001-069912' '070001-079912'
 '080001-089912' '090001-099912' '100001-109912' '110001-119912'
 '120001-129912' '130001-139912' '140001-149912' '150001-159912'
 '160001-169912' '170001-179912' '180001-189912' '190001-199912'
 '200001-209912' '210001-220012' '185001-200512' '192001-200512'
 '200601-208012' '208101-210012' '200601-210012']




**NOTE:**
    
- There are **524** files. The spreadsheet mentions **620** files. 